In [1]:

import scanpy as sc
import matplotlib.pyplot as plt
import sys
import os
import numpy as np
import pandas as pd
import anndata as ad
import scanpy as sc
import networkx as nx
import torch
import sklearn.preprocessing
import episcanpy as epi
import sys
import os
import numpy as np
import pandas as pd
import anndata as ad
import scanpy as sc
# import networkx as nx # No longer needed for training script itself
import torch


adata_ATAC=sc.read("/data/cabins/sccancer/hhy/TI/Fallopian_tube_ATAC.h5ad")
adata_RNA= sc.read("/data/cabins/sccancer/hhy/TI/Fallopian_tube_RNA.h5ad")

In [2]:
adata_ATAC_NK = adata_ATAC[adata_ATAC.obs["cell_type"] == 'pericyte',:]
samples_to_keep = ['55-year-old stage', '61-year-old stage', '62-year-old stage', '65-year-old stage']
mask = adata_ATAC_NK.obs['development_stage'].isin(samples_to_keep)
adata_ATAC_NK_Age = adata_ATAC_NK[mask].copy()

In [3]:
adata_RNA_NK = adata_RNA[adata_RNA.obs["cell_type"] == 'pericyte',:]
samples_to_keep = ['55-year-old stage', '61-year-old stage', '62-year-old stage', '65-year-old stage']
mask = adata_RNA_NK.obs['development_stage'].isin(samples_to_keep)
adata_RNA_NK_Age = adata_RNA_NK[mask].copy()

In [19]:
adata_ATAC_NK.obs["cell_type"]

P3203A_AAACGAAAGTTCAAGA-1    pericyte
P3203A_AAACGAAAGTTCAAGA-2    pericyte
P3203A_AAACTCGGTAACGGCA-1    pericyte
P3203A_AAACTCGGTAACGGCA-2    pericyte
P3203A_AAACTGCGTATATGGA-1    pericyte
                               ...   
P3369I_TGCTCACCACACATGT-1    pericyte
P3369I_TGGCAATGTGAGTAAT-1    pericyte
P3369I_TGGCCTTCACATATCG-1    pericyte
P3369I_TTAGGTGCACACTAGT-1    pericyte
P3369I_TTCTGTAAGATTGACA-1    pericyte
Name: cell_type, Length: 6658, dtype: category
Categories (1, object): ['pericyte']

In [4]:
replacement_map = {
    '55-year-old stage': '55',
    '61-year-old stage': '61',
    '62-year-old stage': '62',
    '65-year-old stage': '65'
}
adata_ATAC_NK_Age.obs['age'] = adata_ATAC_NK_Age.obs['development_stage'].replace(replacement_map)
adata_RNA_NK_Age.obs['age'] = adata_RNA_NK_Age.obs['development_stage'].replace(replacement_map)

In [5]:
sc.pp.filter_cells(adata_RNA_NK_Age, min_genes=100)
sc.pp.filter_genes(adata_RNA_NK_Age, min_cells=3)
# Saving count data
adata_RNA_NK_Age.layers["counts"] = adata_RNA_NK_Age.X.copy()
# Normalizing to median total counts
sc.pp.normalize_total(adata_RNA_NK_Age)
# Logarithmize the data
sc.pp.log1p(adata_RNA_NK_Age)
sc.pp.highly_variable_genes(adata_RNA_NK_Age, n_top_genes=2000)
adata_RNA_NK_Age = adata_RNA_NK_Age[:,adata_RNA_NK_Age.var['highly_variable'] ==True]
fpeak=0.03
epi.pp.binarize(adata_ATAC_NK_Age)
epi.pp.filter_features(adata_ATAC_NK_Age, min_cells=np.ceil(adata_ATAC_NK_Age.shape[0])*fpeak)

In [6]:
# -*- coding: utf-8 -*-
"""
Training script for CGLUE-SOE-OT model using real (preprocessed)
scRNA-seq and scATAC-seq data with ordered age labels.

This script assumes the modified CGLUE-SOE-OT model files
(e.g., cglue_soe_ot.py, cglue_soe_components_ot.py) are
accessible within the scglue package structure or Python path.
"""

import sys
import os
import numpy as np
import pandas as pd
import anndata as ad
import scanpy as sc
# import networkx as nx # No longer needed for training script itself
import torch

# --- Add project root to path if necessary ---
# Adjust this path if your script is located elsewhere relative to the scglue package
# project_root = os.path.abspath(os.path.join(os.path.dirname(__file__), '..'))
# if project_root not in sys.path:
#     sys.path.insert(0, project_root)

try:
    # Import the modified model and config function
    import scglue
    from scglue.models.cglue_soe_ot import CGLUESOE_OT_Model, configure_dataset_cglue_soe
    # AnnDatasetWithLabels is still needed, GraphDataset is not
    from scglue.models.data import AnnDatasetWithLabels
    # from scglue.utils import config as scglue_config # Optional config import
except ImportError as e:
    print(f"Error importing modified scglue components: {e}")
    print("Please ensure the modified scglue package (with CGLUE-SOE-OT) is installed or accessible.")
    sys.exit(1)
except AttributeError as e:
    print(f"Error importing specific components (check file/class names): {e}")
    print("Ensure 'CGLUESOE_OT_Model' and 'configure_dataset_cglue_soe' exist in 'scglue.models.cglue_soe_ot'.")
    sys.exit(1)


# --- Configuration ---
SEED = 0

torch.manual_seed(SEED)
np.random.seed(SEED)
SAVE_DIR = "cglue_soe_ot_Fallopian_tube_pericyte_output" # Changed save directory name
os.makedirs(SAVE_DIR, exist_ok=True)

In [7]:
# --- 1. Load Your Real Data ---
print("Loading real data...")

atac_adata= adata_ATAC_NK_Age
rna_adata = adata_RNA_NK_Age
adatas = {"rna": rna_adata, "atac": atac_adata}

# --- 2. Preprocessing Steps ---
print("Preprocessing data...")

# ** Crucial: Ensure 'age' column is ordered categorical **
# Define the correct order
age_order = ['55', '61', '62','65']
label_col = 'age' # MODIFY AS NEEDED
try:
    for k in adatas:
        if 'age' not in adatas[k].obs:
            raise ValueError(f"'age' column not found in adata '{k}'.")
        adatas[k].obs['age'] = pd.Categorical(
            adatas[k].obs['age'], categories=age_order, ordered=True
        )
        # Check if any values became NaN after conversion (means original values didn't match categories)
        if adatas[k].obs['age'].isnull().any():
             print(f"Warning: Some 'age' values in adata '{k}' did not match the expected categories {age_order} and resulted in NaNs.")
             # Optional: Handle NaNs, e.g., remove cells or raise error
             # adatas[k] = adatas[k][adatas[k].obs['age'].notna()].copy()
except Exception as e:
    print(f"Error processing 'age' column: {e}")
    sys.exit(1)


Loading real data...
Preprocessing data...


In [8]:
atac_adata.var['highly_variable'] = True

In [9]:

# --- 3. Define Vertices (No Graph Construction Needed) ---
# The model still needs the list of all unique features across datasets
# for the feature embedding layer.
print("Defining feature vertices...")
vertices = pd.Index(rna_adata.var_names.tolist() + atac_adata.var_names.tolist()).unique()
print(f"Total unique features (vertices): {len(vertices)}")


Defining feature vertices...
Total unique features (vertices): 17207


In [13]:

# --- 4. Configure Datasets for CGLUE-SOE-OT ---
print("Configuring datasets for CGLUE-SOE-OT...")
# ** Choose appropriate prob_model based on your preprocessing **
# 'Normal' or 'ZILN' for log-normalized RNA.
# 'Normal', 'ZILN', or 'Bernoulli' for ATAC depending on preprocessing.
try:
    configure_dataset_cglue_soe( # Use the same config function name
        adatas["rna"],
        prob_model="Normal", # Adjust if needed (e.g., "NB" if using raw counts)
        use_highly_variable=True, # Use selected HVGs
        use_label=label_col, # Use the ordered label column
        use_layer=None, # Assuming preprocessed data is in .X
        use_rep=None,   # Or specify a representation like 'X_pca' if desired
        use_obs_names=False # Data is unpaired
    )
    configure_dataset_cglue_soe( # Use the same config function name
        adatas["atac"],
        prob_model="Bernoulli", # Adjust based on ATAC preprocessing
        use_highly_variable=True, # Use selected HVGs
        use_label=label_col, # Use the ordered label column
        use_layer=None, # Assuming preprocessed data is in .X
        use_rep=None,   # Or specify a representation like 'X_lsi' if desired
        use_obs_names=False # Data is unpaired
    )
    print("Datasets configured.")
except Exception as e:
    print(f"Error configuring datasets: {e}")
    sys.exit(1)


[WARNING] CGLUESOE_OT_Model: `configure_dataset` already called. Overwriting!


Configuring datasets for CGLUE-SOE-OT...
[INFO] CGLUESOE_OT_Model: Dataset configured for CGLUE-SOE-OT using label 'age'.


[WARNING] CGLUESOE_OT_Model: `configure_dataset` already called. Overwriting!


[INFO] CGLUESOE_OT_Model: Dataset configured for CGLUE-SOE-OT using label 'age'.
Datasets configured.


In [14]:


# --- 5. Initialize CGLUE-SOE-OT Model ---
print("Initializing CGLUE-SOE-OT model...")
try:
    model = CGLUESOE_OT_Model(
        adatas=adatas,
        x2u_h_depth=2, 
        x2u_h_dim=256,                 
        u2x_h_depth=2, 
        u2x_h_dim=256, 
        dropout=0.2,
        vertices=vertices.tolist(), # Pass the combined feature list
        latent_dim=50, # Example latent dimension, adjust as needed
        random_seed=SEED
    )
    print("Model initialized.")
except Exception as e:
    print(f"Error initializing model: {e}")
    sys.exit(1)

# --- 6. Compile Model ---
print("Compiling model...")
Times = '1'
lam_kl=0.1           # Weight for KL divergence regularization 调低有利于聚类更紧密
lam_data=1.0         # Weight for data reconstruction loss 调高有利于聚类更紧密
lam_triplet=500      # Weight for triplet loss (NEEDS TUNING)
lam_ot=5.0           # Weight for Optimal Transport loss (NEEDS TUNING)
triplet_margin=5   # Margin for triplet loss (NEEDS TUNING)有可能数据本身就是按照时间序列排序的
ot_epsilon=0.1       # OT Sinkhorn regularization (NEEDS TUNING) 较小的值 (如当前的 0.1) 匹配更精确，但也可能对噪声更敏感
ot_tau=1.0          # OT Sinkhorn unbalancedness (NEEDS TUNING)
ot_max_iter=100

max_epochs=500
# ** Tune hyperparameters, especially loss weights **
try:
    model.compile(
        lr=1e-3,              # Learning rate for the single optimizer
        # lam_graph removed
        # lam_align removed
        # dsc_lr removed
        # train_ratio removed
        lam_kl=lam_kl,           # Weight for KL divergence regularization 调低有利于聚类更紧密
        lam_data=lam_data,         # Weight for data reconstruction loss 调高有利于聚类更紧密
        lam_triplet=lam_triplet,      # Weight for triplet loss (NEEDS TUNING)
        lam_ot=lam_ot,           # Weight for Optimal Transport loss (NEEDS TUNING)
        triplet_margin=triplet_margin,   # Margin for triplet loss (NEEDS TUNING)有可能数据本身就是按照时间序列排序的
        ot_epsilon=ot_epsilon,       # OT Sinkhorn regularization (NEEDS TUNING) 较小的值 (如当前的 0.1) 匹配更精确，但也可能对噪声更敏感
        ot_tau=ot_tau,           # OT Sinkhorn unbalancedness (NEEDS TUNING)
        ot_max_iter=ot_max_iter,      # OT Sinkhorn iterations
         #weight_decay=1e-5   # Example: Add optimizer args directly
    )
    print("Model compiled.")
except Exception as e:
    print(f"Error compiling model: {e}")
    sys.exit(1)

# --- 7. Train Model ---
print("Fitting model...")
# Fit the model (No GraphDataset needed)
# Adjust training parameters as needed
try:
    model.fit(
        adatas=adatas,            # Pass adatas dictionary
        # graph removed
        val_split=0.2,            # Use 10% of data for validation
        data_batch_size=256,      # Adjust based on memory
        # graph_batch_size removed
        max_epochs=max_epochs,           # Set a reasonable number of epochs
        patience=35,              # Early stopping patience
        reduce_lr_patience=20,    # LR scheduling patience
        wait_n_lrs=3,             # Wait for LR reductions before stopping
        num_workers=0,            # Set > 0 for parallel data loading if needed
        directory=SAVE_DIR        # Directory for logs and checkpoints
    )
    print("Model fitting finished.")
except Exception as e:
    print(f"An error occurred during model fitting: {e}")
    import traceback
    traceback.print_exc()
    sys.exit(1)




Initializing CGLUE-SOE-OT model...
Model initialized.
Compiling model...
Model compiled.
Fitting model...


[INFO] CGLUESOE_OT_Trainer: [Epoch 10] Train={'x_rna_nll': '539.5952', 'x_rna_kl': '54.5226', 'x_rna_elbo': '545.0474', 'x_atac_nll': '9080.1427', 'x_atac_kl': '65.4728', 'x_atac_elbo': '9086.6900', 'triplet_loss': '0.0380', 'ot_loss': '53.6907', 'total_loss': '9919.2143'}, Val={'x_rna_nll': '500.4149', 'x_rna_kl': '54.8820', 'x_rna_elbo': '505.9031', 'x_atac_nll': '8951.0394', 'x_atac_kl': '60.3912', 'x_atac_elbo': '8957.0784', 'triplet_loss': '2.3547', 'ot_loss': '32.5359', 'total_loss': '10803.0322'}
[INFO] CGLUESOE_OT_Trainer: [Epoch 20] Train={'x_rna_nll': '-162.2330', 'x_rna_kl': '86.6351', 'x_rna_elbo': '-153.5695', 'x_atac_nll': '7971.5126', 'x_atac_kl': '106.1100', 'x_atac_elbo': '7982.1236', 'triplet_loss': '0.0246', 'ot_loss': '44.6486', 'total_loss': '8064.1136'}, Val={'x_rna_nll': '-200.3537', 'x_rna_kl': '84.1604', 'x_rna_elbo': '-191.9376', 'x_atac_nll': '7920.2801', 'x_atac_kl': '101.1894', 'x_atac_elbo': '7930.3991', 'triplet_loss': '1.4768', 'ot_loss': '25.6769', 'tot

[INFO] LRScheduler: Learning rate reduction: step 4
[INFO] CGLUESOE_OT_Trainer: [Epoch 170] Train={'x_rna_nll': '-2691.4753', 'x_rna_kl': '176.3187', 'x_rna_elbo': '-2673.8434', 'x_atac_nll': '7432.5348', 'x_atac_kl': '206.9146', 'x_atac_elbo': '7453.2263', 'triplet_loss': '0.0005', 'ot_loss': '5.6505', 'total_loss': '4807.8805'}, Val={'x_rna_nll': '-2454.7768', 'x_rna_kl': '174.8369', 'x_rna_elbo': '-2437.2931', 'x_atac_nll': '7536.0837', 'x_atac_kl': '193.3486', 'x_atac_elbo': '7555.4185', 'triplet_loss': '1.0653', 'ot_loss': '8.5992', 'total_loss': '5693.7961'}
[INFO] CGLUESOE_OT_Trainer: [Epoch 180] Train={'x_rna_nll': '-2680.9408', 'x_rna_kl': '176.2398', 'x_rna_elbo': '-2663.3168', 'x_atac_nll': '7454.9196', 'x_atac_kl': '207.1386', 'x_atac_elbo': '7475.6335', 'triplet_loss': '0.0030', 'ot_loss': '5.2979', 'total_loss': '4840.3303'}, Val={'x_rna_nll': '-2454.6084', 'x_rna_kl': '174.4728', 'x_rna_elbo': '-2437.1611', 'x_atac_nll': '7535.1558', 'x_atac_kl': '193.6062', 'x_atac_elbo

[INFO] CGLUESOE_OT_Trainer: [Epoch 330] Train={'x_rna_nll': '-2695.0362', 'x_rna_kl': '176.0864', 'x_rna_elbo': '-2677.4276', 'x_atac_nll': '7436.0223', 'x_atac_kl': '207.0018', 'x_atac_elbo': '7456.7225', 'triplet_loss': '0.0050', 'ot_loss': '5.5435', 'total_loss': '4809.4979'}, Val={'x_rna_nll': '-2454.7057', 'x_rna_kl': '174.7106', 'x_rna_elbo': '-2437.2346', 'x_atac_nll': '7534.4268', 'x_atac_kl': '193.3644', 'x_atac_elbo': '7553.7632', 'triplet_loss': '1.0407', 'ot_loss': '8.5728', 'total_loss': '5679.7367'}
[INFO] CGLUESOE_OT_Trainer: [Epoch 340] Train={'x_rna_nll': '-2694.4455', 'x_rna_kl': '176.4118', 'x_rna_elbo': '-2676.8043', 'x_atac_nll': '7448.9387', 'x_atac_kl': '206.7000', 'x_atac_elbo': '7469.6087', 'triplet_loss': '0.0029', 'ot_loss': '5.3370', 'total_loss': '4820.9387'}, Val={'x_rna_nll': '-2454.3047', 'x_rna_kl': '174.3212', 'x_rna_elbo': '-2436.8726', 'x_atac_nll': '7534.3313', 'x_atac_kl': '193.7011', 'x_atac_elbo': '7553.7014', 'triplet_loss': '0.9844', 'ot_loss':

[INFO] CGLUESOE_OT_Trainer: [Epoch 490] Train={'x_rna_nll': '-2682.2083', 'x_rna_kl': '176.0240', 'x_rna_elbo': '-2664.6059', 'x_atac_nll': '7443.0309', 'x_atac_kl': '207.1771', 'x_atac_elbo': '7463.7487', 'triplet_loss': '0.0058', 'ot_loss': '5.5665', 'total_loss': '4829.8674'}, Val={'x_rna_nll': '-2454.4592', 'x_rna_kl': '174.3090', 'x_rna_elbo': '-2437.0283', 'x_atac_nll': '7533.3564', 'x_atac_kl': '193.5717', 'x_atac_elbo': '7552.7136', 'triplet_loss': '1.0126', 'ot_loss': '7.5677', 'total_loss': '5659.8073'}
[INFO] CGLUESOE_OT_Trainer: [Epoch 500] Train={'x_rna_nll': '-2685.4923', 'x_rna_kl': '176.0640', 'x_rna_elbo': '-2667.8860', 'x_atac_nll': '7424.3940', 'x_atac_kl': '206.4751', 'x_atac_elbo': '7445.0415', 'triplet_loss': '0.0030', 'ot_loss': '5.2868', 'total_loss': '4805.0764'}, Val={'x_rna_nll': '-2453.8670', 'x_rna_kl': '174.1122', 'x_rna_elbo': '-2436.4558', 'x_atac_nll': '7534.4141', 'x_atac_kl': '193.4536', 'x_atac_elbo': '7553.7595', 'triplet_loss': '1.0447', 'ot_loss':

In [15]:
# --- 8. Post-Training Analysis ---
print("Extracting embeddings...")
try:
    # Use a different key for the new model's embeddings
    embedding_key = "X_cgluesoe_ot"
    rna_adata.obsm[embedding_key] = model.encode_data("rna", rna_adata)
    atac_adata.obsm[embedding_key] = model.encode_data("atac", atac_adata)

    # Get feature embeddings from the nn.Embedding layer
    feature_embeddings = model.get_feature_embeddings()
    feature_embeddings_df = pd.DataFrame(feature_embeddings, index=model.vertices)

    print("RNA embeddings shape:", rna_adata.obsm[embedding_key].shape)
    print("ATAC embeddings shape:", atac_adata.obsm[embedding_key].shape)
    print("Feature embeddings shape:", feature_embeddings_df.shape)

    # --- Optional: Save model and embeddings ---
    final_model_path = os.path.join(SAVE_DIR, "cglue_soe_ot_final_model.dill")
    print(f"Saving final model to {final_model_path}")
    model.save(final_model_path)

    rna_save_path = os.path.join(SAVE_DIR, "rna_adata_with_embeddings.h5ad")
    atac_save_path = os.path.join(SAVE_DIR, "atac_adata_with_embeddings.h5ad")
    feat_save_path = os.path.join(SAVE_DIR, "feature_embeddings.csv")

    # print(f"Saving RNA AnnData to {rna_save_path}")
    # rna_adata.write_h5ad(rna_save_path)
    # print(f"Saving ATAC AnnData to {atac_save_path}")
    # atac_adata.write_h5ad(atac_save_path)
    # print(f"Saving feature embeddings to {feat_save_path}")
    # feature_embeddings_df.to_csv(feat_save_path)


    # --- Optional: Visualization (UMAP) ---
    print("Generating UMAP visualization...")
    # Combine embeddings and obs for visualization
    all_embeddings = np.concatenate([rna_adata.obsm[embedding_key], atac_adata.obsm[embedding_key]], axis=0)
    all_obs = pd.concat([
        rna_adata.obs[[label_col]].assign(modality='rna'), # Use label_col
        atac_adata.obs[[label_col]].assign(modality='atac') # Use label_col
    ], axis=0)
    # Create combined obs_names for the visualization AnnData
    vis_obs_names = rna_adata.obs_names.tolist() + atac_adata.obs_names.tolist()
    if len(vis_obs_names) != len(all_obs): # Basic check
         print(f"Warning: Length mismatch between obs_names ({len(vis_obs_names)}) and combined obs ({len(all_obs)}). Using default index.")
         vis_obs_names = pd.RangeIndex(start=0, stop=len(all_obs), step=1)


    vis_adata = ad.AnnData(all_embeddings, dtype=all_embeddings.dtype) # Ensure dtype is set
    vis_adata.obs = all_obs
    vis_adata.obs_names = vis_obs_names # Assign combined names
    vis_adata.obsm['X_cgluesoe_ot'] = all_embeddings # Store for scanpy functions

    print("Running neighbors and UMAP...")
    sc.pp.neighbors(vis_adata, use_rep='X_cgluesoe_ot', n_neighbors=15, random_state=SEED)
    sc.tl.umap(vis_adata, min_dist=0.3, random_state=SEED) # Adjust min_dist for spread

    print("Plotting UMAP...")
    umap_save_path = f'_cglue_soe_ot_Fallopian_pericyte_{Times}_results.png' # Updated filename
    sc.pl.umap(
        vis_adata,
        color=['modality', label_col], # Color by label_col
        title=['UMAP by Modality', f'UMAP by {label_col.capitalize()}'],
        save=umap_save_path,
        show=False # Set to True to display interactively if possible
    )
    print(f"UMAP plots saved to figures/ directory ({umap_save_path}).")

except Exception as e:
    print(f"An error occurred during post-training analysis: {e}")
    import traceback
    traceback.print_exc()

print("Script finished.")

Extracting embeddings...
RNA embeddings shape: (9324, 50)
ATAC embeddings shape: (6658, 50)
Feature embeddings shape: (17207, 50)
Saving final model to cglue_soe_ot_Fallopian_tube_pericyte_output/cglue_soe_ot_final_model.dill
[INFO] CGLUESOE_OT_Model: Model state saved to cglue_soe_ot_Fallopian_tube_pericyte_output/cglue_soe_ot_final_model.dill
Generating UMAP visualization...
Running neighbors and UMAP...


Plotting UMAP...
UMAP plots saved to figures/ directory (_cglue_soe_ot_Fallopian_pericyte_1_results.png).
Script finished.


In [16]:

print("Plotting loss curves...")
try:
    # Define where to save the plot
    loss_plot_path = os.path.join(SAVE_DIR, f"cglue_soe_ot_loss_curves_Fallopian_pericyte_{Times}.png")
    # Call the plotting method
    model.plot_loss_curves(save_path=loss_plot_path)
    print(f"Loss curves saved to {loss_plot_path}")
except Exception as e:
    print(f"An error occurred during loss plotting: {e}")
    import traceback
    traceback.print_exc()


Plotting loss curves...
[INFO] CGLUESOE_OT_Model: Loss curves saved to cglue_soe_ot_Fallopian_tube_pericyte_output/cglue_soe_ot_loss_curves_Fallopian_pericyte_1.png
Loss curves saved to cglue_soe_ot_Fallopian_tube_pericyte_output/cglue_soe_ot_loss_curves_Fallopian_pericyte_1.png


In [17]:
import csv
import pickle
# 定义你的超参数 (与上面相同)
hyperparameters = {
    "lam_kl": lam_kl,
    "lam_data": lam_data,
    "lam_triplet": lam_triplet,
    "lam_ot": lam_ot,
    "triplet_margin": triplet_margin,
    "ot_epsilon": ot_epsilon,
    "ot_tau": ot_tau,
    "ot_max_iter": ot_max_iter,
    "max_epochs": max_epochs
}

# 定义要保存的PKL文件名
csv_filename = os.path.join(SAVE_DIR, "hyperparameters_pericyte.csv")

# 写入CSV文件
try:
    with open(csv_filename, 'a', newline='') as csvfile:
        # 定义CSV写入器
        writer = csv.writer(csvfile)

        # 写入表头 (可选，但推荐)
        writer.writerow([f'ParameterName_{Times}', 'Value'])

        # 遍历字典并写入每一行
        for name, value in hyperparameters.items():
            writer.writerow([name, value])
    print(f"超参数已成功保存到 {csv_filename}")
except IOError:
    print(f"错误：无法写入文件 {csv_filename}")

超参数已成功保存到 cglue_soe_ot_Fallopian_tube_pericyte_output/hyperparameters_pericyte.csv
